In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
plt.rcParams['font.family'] = 'Gulim'



RuntimeError: module compiled against API version 0xe but this version of numpy is 0xd

In [2]:
from workalendar.asia import SouthKorea
import pendulum

In [3]:
train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')

## 전처리
- 일자에서 월과 일을 분리
- 요일을 레이블 인코딩화(EDA로 요일의 중요도 순 파악)
- 월 별, 일 별 중식 석식 수요 차이 파악

In [4]:
train['월'] = pd.DatetimeIndex(train['일자']).month
test['월'] = pd.DatetimeIndex(test['일자']).month
train['주'] = pd.DatetimeIndex(train['일자']).week
test['주'] = pd.DatetimeIndex(test['일자']).week
train['일'] = pd.DatetimeIndex(train['일자']).day
test['일'] = pd.DatetimeIndex(test['일자']).day

train['출근'] = train['본사정원수']-(train['본사휴가자수']+train['본사출장자수']+train['현본사소속재택근무자수'])
train['휴가비율'] = train['본사휴가자수']/train['본사정원수']
train['출장비율'] = train['본사출장자수']/train['본사정원수']
train['야근비율'] = train['본사시간외근무명령서승인건수']/train['출근']
train['재택비율'] = train['현본사소속재택근무자수']/train['본사정원수']

test['출근'] = test['본사정원수']-(test['본사휴가자수']+test['본사출장자수']+test['현본사소속재택근무자수'])
test['휴가비율'] = test['본사휴가자수']/test['본사정원수']
test['출장비율'] = test['본사출장자수']/test['본사정원수']
test['야근비율'] = test['본사시간외근무명령서승인건수']/test['출근']
test['재택비율'] = test['현본사소속재택근무자수']/test['본사정원수']

train['식사가능자수'] = train['본사정원수'] - train['본사휴가자수'] - train['현본사소속재택근무자수']
test['식사가능자수'] = test['본사정원수'] - test['본사휴가자수'] - test['현본사소속재택근무자수']

# train['중식참여율'] = train['중식계'] / train['식사가능자수']

<ipython-input-4-b11c832c5082>:3: FutureWarning: weekofyear and week have been deprecated, please use DatetimeIndex.isocalendar().week instead, which returns a Series.  To exactly reproduce the behavior of week and weekofyear and return an Index, you may call pd.Int64Index(idx.isocalendar().week)
  train['주'] = pd.DatetimeIndex(train['일자']).week
<ipython-input-4-b11c832c5082>:4: FutureWarning: weekofyear and week have been deprecated, please use DatetimeIndex.isocalendar().week instead, which returns a Series.  To exactly reproduce the behavior of week and weekofyear and return an Index, you may call pd.Int64Index(idx.isocalendar().week)
  test['주'] = pd.DatetimeIndex(test['일자']).week


In [5]:
month_rank4dinner = {
    1: 11,
    2: 2,
    3: 1,
    4: 4,
    5: 7,
    6: 6,
    7: 10,
    8: 8,
    9: 5,
    10: 3,
    11: 9,
    12: 12
}
train['월(석식)'] = train['월'].map(month_rank4dinner)
test['월(석식)'] = test['월'].map(month_rank4dinner)

month_rank4lunch = {
    1: 3,
    2: 1,
    3: 2,
    4: 6,
    5: 7,
    6: 8,
    7: 10,
    8: 9,
    9: 5,
    10: 4,
    11: 11,
    12: 12
}
train['월(중식)'] = train['월'].map(month_rank4lunch)
test['월(중식)'] = test['월'].map(month_rank4lunch)

weekday_rank4dinner = {
    '월': 1,
    '화': 2,
    '수': 4,
    '목': 3,
    '금': 5,
}

weekday_rank4lunch = {
    '월': 1,
    '화': 2,
    '수': 3,
    '목': 4,
    '금': 5,
}

train['요일(석식)'] = train['요일'].map(weekday_rank4dinner)
test['요일(석식)'] = test['요일'].map(weekday_rank4dinner)

train['요일(중식)'] = train['요일'].map(weekday_rank4lunch)
test['요일(중식)'] = test['요일'].map(weekday_rank4lunch)

In [6]:
rank = pd.DataFrame(range(1,53))
week_rank_lunch = pd.pivot_table(train,values='중식계',index='주').sort_values(by='중식계').reset_index().drop('중식계',axis=1)
week_rank_dinner = pd.pivot_table(train,values='석식계',index='주').sort_values(by='석식계').reset_index().drop('석식계',axis=1)


week_rank4lunch = {}
for i in range(len(rank)):
    week_rank4lunch[week_rank_lunch['주'][i]] = rank[0][i]


week_rank4dinner = {}
for i in range(len(rank)):
    week_rank4dinner[week_rank_dinner['주'][i]] = rank[0][i]
    
    
train['주(중식)'] = train['주'].map(week_rank4lunch)
test['주(중식)'] = test['주'].map(week_rank4lunch)

train['주(석식)'] = train['주'].map(week_rank4dinner)
test['주(석식)'] = test['주'].map(week_rank4dinner)

In [7]:
def is_holiday(date):
    holidays = list(map(str, pd.Series(np.array(SouthKorea().holidays(int(date[:4])))[:, 0])))
    
    yesterday = str(np.datetime64(date) - 1)
    tomorrow = str(np.datetime64(date) + 1)

    if tomorrow in holidays and yesterday in holidays:
        return 3
    if tomorrow in holidays:
        return 2
    elif yesterday in holidays:
        return 1
    else : 
        return 0

def week_of_month(x):
    dt = pendulum.parse(x)
    
    wom = dt.week_of_month
    if wom < 0:
        wom += 52
    return wom
    

df = pd.concat([train[['본사정원수', '일자']], test[['본사정원수', '일자']]])
df['년월'] = df['일자'].apply(lambda x : x[:7])
df = df[['년월', '본사정원수']].groupby(by=['년월'], as_index=False).mean()

def member_change(date):
    this_month = date[:7]
    last_month = str(np.datetime64(this_month) - 1)
    
    this_month_member = int(df[df['년월'] == this_month]['본사정원수'])
    last_month_member = int(df[df['년월'] == last_month]['본사정원수'])
    
    
    return  this_month_member - last_month_member

train['공휴일전후'] = train['일자'].apply(is_holiday)
test['공휴일전후'] = test['일자'].apply(is_holiday)

train['몇주차'] = train['일자'].apply(week_of_month)
test['몇주차'] = test['일자'].apply(week_of_month)

train = train[train['일자'] > '2016-03']
train['인원변화'] = train['일자'].apply(member_change)
test['인원변화'] = test['일자'].apply(member_change)

## 공휴일 변수 생성

In [8]:
train.columns

Index(['일자', '요일', '본사정원수', '본사휴가자수', '본사출장자수', '본사시간외근무명령서승인건수',
       '현본사소속재택근무자수', '조식메뉴', '중식메뉴', '석식메뉴', '중식계', '석식계', '월', '주', '일',
       '출근', '휴가비율', '출장비율', '야근비율', '재택비율', '식사가능자수', '월(석식)', '월(중식)',
       '요일(석식)', '요일(중식)', '주(중식)', '주(석식)', '공휴일전후', '몇주차', '인원변화'],
      dtype='object')

In [9]:
# 메뉴 변수 없이 사용할떄 해당 코드 사용['공휴일전후', '몇주차', '인원변화']

lunch_train = train[['공휴일전후', '몇주차', '인원변화', '요일(중식)','월(중식)','일','주(중식)','출근', '휴가비율', '출장비율', '야근비율', '재택비율','본사출장자수','본사휴가자수','식사가능자수',]]
lunch_test = test[['공휴일전후', '몇주차', '인원변화', '요일(중식)','월(중식)','일','주(중식)','출근', '휴가비율', '출장비율', '야근비율', '재택비율','본사출장자수','본사휴가자수','식사가능자수']]

dinner_train= train[['공휴일전후', '몇주차', '인원변화', '요일(석식)','월(석식)','일','주(석식)','출근', '휴가비율', '출장비율', '야근비율', '재택비율','본사출장자수','본사휴가자수','식사가능자수']]
dinner_test = test[['공휴일전후', '몇주차', '인원변화', '요일(석식)','월(석식)','일','주(석식)','출근', '휴가비율', '출장비율', '야근비율', '재택비율','본사출장자수','본사휴가자수','식사가능자수']]

In [10]:
lunch_train.columns

Index(['공휴일전후', '몇주차', '인원변화', '요일(중식)', '월(중식)', '일', '주(중식)', '출근', '휴가비율',
       '출장비율', '야근비율', '재택비율', '본사출장자수', '본사휴가자수', '식사가능자수'],
      dtype='object')

In [11]:
print(lunch_train.shape)
print(lunch_test.shape)

(1187, 15)
(50, 15)


In [12]:
print(dinner_train.shape)
print(dinner_test.shape)

(1187, 15)
(50, 15)


In [13]:
cat_features = [f for f in lunch_train.columns if lunch_train[f].dtype == 'object']

def column_index(df, cat_features):
    cols = df.columns.values
    sidx = np.argsort(cols)
    return sidx[np.searchsorted(cols, cat_features, sorter=sidx)]

cat_features_idx = column_index(lunch_train, cat_features)    
print("Cat features are: %s" % [f for f in cat_features])
print(cat_features_idx)

Cat features are: []
[]


In [14]:
y_lunch = train[['중식계']]
y_dinner = train[['석식계']]

In [15]:
lunch_train.columns

Index(['공휴일전후', '몇주차', '인원변화', '요일(중식)', '월(중식)', '일', '주(중식)', '출근', '휴가비율',
       '출장비율', '야근비율', '재택비율', '본사출장자수', '본사휴가자수', '식사가능자수'],
      dtype='object')

#### 분포 확인 및 분포 조정

# 중식 예측모델

In [17]:
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_absolute_error as MAE
from IPython.display import clear_output
import sklearn
from sklearn.model_selection import train_test_split, KFold

In [18]:
lunch_param = {'reg_alpha': 1.622564509910422e-05, 'reg_lambda': 0.0727906280606794, 'max_depth': 12, 'num_leaves': 7, 'colsample_bytree': 0.7041252632987905, 'subsample': 0.9930902085857553, 'subsample_freq': 5, 'min_child_samples': 24, 'max_bin': 437}
dinner_param = {'reg_alpha': 1.718625207505066e-06, 'reg_lambda': 0.03536226882919215, 'max_depth': 13, 'num_leaves': 149, 'colsample_bytree': 0.7238257801656021, 'subsample': 0.9792806922458114, 'subsample_freq': 3, 'min_child_samples': 26, 'max_bin': 423}

In [40]:
drop_col = [[c] for c in  list(lunch_train.columns)]

In [56]:
drop_columns = [[]] + drop_col
ks = range(2,16,2)

In [57]:
random_seed = 0
pred_lunch = np.array([0]*50).astype(np.float64)
y_lunch = np.array(y_lunch).reshape(-1,1)

for dc in drop_columns:
    for n_fold in ks:
        
        kf = KFold(n_splits=n_fold, random_state=42, shuffle=True)
        log_scores = []

        for train_index, test_index in kf.split(lunch_train):
            X_train, X_valid = lunch_train.iloc[train_index], lunch_train.iloc[test_index]
            y_train, y_valid = y_lunch[train_index], y_lunch[test_index]

            model = LGBMRegressor(**lunch_param)
            model.fit(
                X_train,
                y_train,
                eval_set=[(X_train, y_train), (X_valid, y_valid)],
                early_stopping_rounds=100,
                verbose=False,
            )
        
            
            pred_lunch += model.predict(lunch_test)/n_fold
            
        clear_output(True)
                
        random_seed += 1

C:\anaconda3\envs\machinelearning\lib\site-packages\lightgbm\basic.py:154: UserWarning: Converting column-vector to 1d array
  _log_warning('Converting column-vector to 1d array')
C:\anaconda3\envs\machinelearning\lib\site-packages\lightgbm\basic.py:154: UserWarning: Converting column-vector to 1d array
  _log_warning('Converting column-vector to 1d array')
C:\anaconda3\envs\machinelearning\lib\site-packages\lightgbm\basic.py:154: UserWarning: Converting column-vector to 1d array
  _log_warning('Converting column-vector to 1d array')
C:\anaconda3\envs\machinelearning\lib\site-packages\lightgbm\basic.py:154: UserWarning: Converting column-vector to 1d array
  _log_warning('Converting column-vector to 1d array')
C:\anaconda3\envs\machinelearning\lib\site-packages\lightgbm\basic.py:154: UserWarning: Converting column-vector to 1d array
  _log_warning('Converting column-vector to 1d array')
C:\anaconda3\envs\machinelearning\lib\site-packages\lightgbm\basic.py:154: UserWarning: Converting c

In [58]:
random_seed = 0
pred_dinner = np.array([0]*50).astype(np.float64)
y_dinner = np.array(y_dinner).reshape(-1,1)
for dc in drop_columns:
    for n_fold in ks:
        
        kf = KFold(n_splits=n_fold, random_state=42, shuffle=True)
        log_scores = []

        for train_index, test_index in kf.split(dinner_train):
            X_train, X_valid = dinner_train.iloc[train_index], dinner_train.iloc[test_index]
            y_train, y_valid = y_dinner[train_index], y_dinner[test_index]

            model = LGBMRegressor(**dinner_param)
            model.fit(
                X_train,
                y_train,
                eval_set=[(X_train, y_train), (X_valid, y_valid)],
                early_stopping_rounds=100,
                verbose=False,
            )
        
            
            pred_dinner += model.predict(dinner_test)/n_fold
            
        clear_output(True)
                
        random_seed += 1

C:\anaconda3\envs\machinelearning\lib\site-packages\lightgbm\basic.py:154: UserWarning: Converting column-vector to 1d array
  _log_warning('Converting column-vector to 1d array')
C:\anaconda3\envs\machinelearning\lib\site-packages\lightgbm\basic.py:154: UserWarning: Converting column-vector to 1d array
  _log_warning('Converting column-vector to 1d array')
C:\anaconda3\envs\machinelearning\lib\site-packages\lightgbm\basic.py:154: UserWarning: Converting column-vector to 1d array
  _log_warning('Converting column-vector to 1d array')
C:\anaconda3\envs\machinelearning\lib\site-packages\lightgbm\basic.py:154: UserWarning: Converting column-vector to 1d array
  _log_warning('Converting column-vector to 1d array')
C:\anaconda3\envs\machinelearning\lib\site-packages\lightgbm\basic.py:154: UserWarning: Converting column-vector to 1d array
  _log_warning('Converting column-vector to 1d array')
C:\anaconda3\envs\machinelearning\lib\site-packages\lightgbm\basic.py:154: UserWarning: Converting c

In [49]:
submission = pd.read_csv('../data/sample_submission.csv')
N = len(ks)*len(drop_columns)

submission.iloc[:,1] = pred_lunch/N
submission.iloc[:,2] = pred_dinner/N
submission.head()

,일자,중식계,석식계
0,2021-01-27,1008.986815,277.212104
1,2021-01-28,921.051015,409.912702
2,2021-01-29,624.959965,206.136461
3,2021-02-01,1299.280654,551.036652
4,2021-02-02,1106.683367,455.684450


In [45]:
def compare_ans(DIR):
    answer = pd.read_csv(DIR)

    lunch_answer = np.array(answer.iloc[:,1])
    dinner_answer = np.array(answer.iloc[:,2])
    
    lunch_MAE = abs(submission.iloc[:,1] - lunch_answer).mean()
    dinner_MAE = abs(submission.iloc[:,2] - dinner_answer).mean()
    
    print("lunch_MAE : ", lunch_MAE)
    print("dinner_MAE : ", dinner_MAE)
    print("total_MAE : ", (lunch_MAE+dinner_MAE)/2)

In [52]:
compare_ans('../submission/20210720_optuna_lgbm.csv')

lunch_MAE :  18.402992541884792
dinner_MAE :  19.76416461265663
total_MAE :  19.08357857727071


In [48]:
answer = pd.read_csv('../submission/20210630_lgbm_autoML.csv')

lunch_answer = np.array(answer.iloc[:,1])
dinner_answer = np.array(answer.iloc[:,2])

abs(pred_lunch/N - lunch_answer).mean(), abs(pred_dinner/N - dinner_answer).mean()

(16.49537088386855, 9.859359321929544)

# 저장

In [47]:
import datetime
today = str(datetime.datetime.now().date()).replace("-","")
print("오늘 날짜 : " + today)

submission.to_csv(f'../submission/{today}_lgbm_knfold_drop.csv', index =False)

오늘 날짜 : 20210721
